## Librerías

In [9]:
%%capture
%run "../../recurrentes.ipynb"

In [10]:
%%capture
%run "../../funciones.ipynb"

In [11]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:
%config SqlMagic.named_parameters = "enabled"



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Creación clase ternaria

In [12]:
# %%sql

# create or replace table competencia_03 as
# with periodos as (
#     select distinct foto_mes from competencia_03_crudo
# ), clientes as (
#     select distinct numero_de_cliente from competencia_03_crudo
# ), todo as (
#     select numero_de_cliente, foto_mes from clientes cross join periodos
# ), clase_ternaria as (
#     select
#         c.*
#         , if(c.numero_de_cliente is null, 0, 1) as mes_0 
#         , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
#         , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
#         , if(mes_2 = 1, 'CONTINUA',
#              if(mes_1 = 0, 'BAJA+1',
#                 if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria 
#     from todo t
#     left join competencia_03_crudo c using (numero_de_cliente, foto_mes)
# ) 

# select * EXCLUDE (mes_0, mes_1, mes_2) 
# from clase_ternaria
# where mes_0 = 1 
# ;


In [13]:
# %%sql 
# SELECT * FROM competencia_03;

In [14]:
# %sql COPY competencia_03 TO 'D:/competencia_03.parquet' (FORMAT 'PARQUET');

# Feature Engineering

In [15]:
%%sql
CREATE OR REPLACE TABLE competencia_03 AS
SELECT * FROM read_parquet(:dataset_clase_ternaria_sin_psi_l);

,Success


In [17]:
paired_variables = [
    ('mtarjeta_visa_consumo', 'mtarjeta_master_consumo'),
    ('ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'),
    ('mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'),
    ('ctarjeta_visa_descuentos', 'ctarjeta_master_descuentos'),
    ('mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos'),
    ('Visa_Fvencimiento', 'Master_Fvencimiento'),
    ('Visa_msaldototal', 'Master_msaldototal'),
    ('Visa_msaldopesos', 'Master_msaldopesos'),
    ('Visa_msaldodolares', 'Master_msaldodolares'),
    ('Visa_mconsumospesos', 'Master_mconsumospesos'),
    ('Visa_mconsumosdolares', 'Master_mconsumosdolares'),
    ('Visa_mlimitecompra', 'Master_mlimitecompra'),
    ('Visa_madelantopesos', 'Master_madelantopesos'),
    ('Visa_madelantodolares', 'Master_madelantodolares'),
    ('Visa_fultimo_cierre', 'Master_fultimo_cierre'),
    ('Visa_mpagado', 'Master_mpagado'),
    ('Visa_mpagospesos', 'Master_mpagospesos'),
    ('Visa_mpagosdolares', 'Master_mpagosdolares'),
    ('Visa_fechaalta', 'Master_fechaalta'),
    ('Visa_mconsumotal', 'Master_mconsumototal'),
    ('Visa_cconsumos', 'Master_cconsumos'),
    ('Visa_cadelantosefectivo', 'Master_cadelantosefectivo'),
    ('Visa_mpagominimo', 'Master_mpagominimo')
]

In [25]:
tarjetas_variables = [
    
    ('ctarjeta_visa_debitos_automaticos', 'ctarjeta_master_debitos_automaticos'),
    ('mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos'),
    ('ctarjeta_visa_descuentos', 'ctarjeta_master_descuentos'),
    ('mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos'),
   
    ('Visa_msaldototal', 'Master_msaldototal'),
    ('Visa_msaldopesos', 'Master_msaldopesos'),
    ('Visa_msaldodolares', 'Master_msaldodolares'),
    ('Visa_mconsumospesos', 'Master_mconsumospesos'),
    ('Visa_mconsumosdolares', 'Master_mconsumosdolares'),
    ('Visa_mlimitecompra', 'Master_mlimitecompra'),
    ('Visa_madelantopesos', 'Master_madelantopesos'),
    ('Visa_madelantodolares', 'Master_madelantodolares'),
    ('Visa_fultimo_cierre', 'Master_fultimo_cierre'),
    ('Visa_mpagado', 'Master_mpagado'),
    ('Visa_mpagospesos', 'Master_mpagospesos'),
    ('Visa_mpagosdolares', 'Master_mpagosdolares'),
    ('Visa_fechaalta', 'Master_fechaalta'),
    
    ('Visa_cconsumos', 'Master_cconsumos'),
    ('Visa_cadelantosefectivo', 'Master_cadelantosefectivo'),
    ('Visa_mpagominimo', 'Master_mpagominimo')
]


In [26]:
paired_columns = []
for visa, master in tarjetas_variables:
    paired_columns.append(f"IFNULL({visa}, 0) + IFNULL ({master},0) AS conjunto_{visa}")

paired_columns_sql = ",\n    ".join(paired_columns)





In [28]:
query = f"""
CREATE OR REPLACE TABLE competencia_03 AS
SELECT *,
    {paired_columns_sql}
FROM competencia_03
"""

%sql {{query}}

,Success


In [29]:
%%sql 


create or replace table competencia_03 as
select
    *,
    (case when Visa_status = 0 then True end) as T_Visa_normal,
    (case when Master_status = 0 then True end) as T_Master_normal
from competencia_03;


create or replace table competencia_03 as
        select *,
        CASE 
        WHEN Visa_status = 6 then 1
        WHEN Visa_status = 7 then 2 
        WHEN Visa_status = 8 then 3
        ELSE NULL
        END as Visa_status
        from competencia_03;

,Success


In [30]:
%%sql

alter table competencia_03 drop column visa_status;
alter table competencia_03 drop column Master_status;

,Success


In [38]:
lista_feature = ['active_quarter',
 'mrentabilidad',
 
 'mcuenta_corriente',
 'mcaja_ahorro',
 'mcuentas_saldo',
 'ctarjeta_debito_transacciones',
 'mautoservicio',
 'mprestamos_prendarios',
 'cprestamos_hipotecarios',
 'mprestamos_hipotecarios',
 'cplazo_fijo',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'cpayroll_trx',
 'mpayroll',
 'mpayroll2',
 'cpayroll2_trx',
 'ccuenta_debitos_automaticos',
 'mcuenta_debitos_automaticos',
 'cpagodeservicios',
 'mpagodeservicios',
 'mpagomiscuentas',
 'cpagomiscuentas',
 'ccajeros_propios_descuentos',
 'mcajeros_propios_descuentos',
 'ccomisiones_mantenimiento',
 'ccomisiones_otras',
 'cforex',
 'cforex_buy',
 'mforex_buy',
 'cforex_sell',
 'mforex_sell',
 'ctransferencias_recibidas',
 'ctransferencias_emitidas',
 'mtransferencias_emitidas',
 'cextraccion_autoservicio',
 'mextraccion_autoservicio',
 'ccheques_depositados',
 'mcheques_depositados',
 'ccheques_emitidos',
 'mcheques_emitidos',
 'ccheques_depositados_rechazados',
 'mcheques_depositados_rechazados',
 'ccheques_emitidos_rechazados',
 'mcheques_emitidos_rechazados',
 'ccallcenter_transacciones',
 'chomebanking_transacciones',
 'ccajas_transacciones',
 'ccajas_consultas',
 'ccajas_depositos',
 'ccajas_extracciones',
 'ccajas_otras',
 'catm_trx',
 'matm',
 'catm_trx_other',
 'matm_other',
 'ctrx_quarter',
 'conjunto_ctarjeta_visa_debitos_automaticos',
 'conjunto_mttarjeta_visa_debitos_automaticos',
 'conjunto_ctarjeta_visa_descuentos',
 'conjunto_mtarjeta_visa_descuentos',
 'conjunto_Visa_msaldototal',
 'conjunto_Visa_msaldopesos',
 'conjunto_Visa_msaldodolares',
 'conjunto_Visa_mconsumospesos',
 'conjunto_Visa_mconsumosdolares',
 'conjunto_Visa_mlimitecompra',
 'conjunto_Visa_madelantopesos',
 'conjunto_Visa_madelantodolares',
 'conjunto_Visa_fultimo_cierre',
 'conjunto_Visa_mpagado',
 'conjunto_Visa_mpagospesos',
 'conjunto_Visa_mpagosdolares',
 'conjunto_Visa_fechaalta',
 'conjunto_Visa_cconsumos',
 'conjunto_Visa_cadelantosefectivo',
 'conjunto_Visa_mpagominimo']



In [39]:
lag_columns = []
for variable in lista_feature:
    lag_columns.append(f"lag({variable}, 1) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_1")
    lag_columns.append(f"lag({variable}, 2) over (partition by numero_de_cliente order by foto_mes) as {variable}_lag_2")


lag_columns_sql = ",\n    ".join(lag_columns)


query = f"""
CREATE OR REPLACE TABLE competencia_03 AS
SELECT *,
    {lag_columns_sql}
FROM competencia_03
"""

%sql {{query}}



,Success


In [40]:
delta_columns = []
for variable in lista_feature:
    nombre_variable = variable+'_delta_1'
    nombre_variable_2 = variable+'_delta_2'
    delta_columns.append(f"{variable} - {variable}_lag_1 as {nombre_variable}")
    delta_columns.append(f"{variable}_lag_1 - {variable}_lag_2 as {nombre_variable_2}")

delta_columns_sql = ",\n    ".join(delta_columns)


query = f'''CREATE OR REPLACE TABLE competencia_03 AS
            select *,
            {delta_columns_sql}
            from competencia_03'''
%sql {{query}}
            

,Success


In [41]:
path_dataset_lags_deltas_y_clase_ternaria = datasets_path_l+r'\competencia_03_lags_deltas_y_clase_ternaria.parquet'
path_dataset_lags_deltas_y_clase_ternaria


'C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_3\\competencia_03_lags_deltas_y_clase_ternaria.parquet'

In [42]:
%sql COPY competencia_03 TO '{{path_dataset_lags_deltas_y_clase_ternaria}}' (FORMAT 'parquet')


,Success


In [ ]:
datita = pd.read_parquet('C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1_UBA_Maestria_DS\\1_Especializacion\\1_Segundo_Semestre\\DMEyF\\datasets\\competencia_2\\competencia_02_lags_deltas_y_clase_ternaria.parquet'
) 
datita.shape

(4735593, 600)